In [1]:
# Imports
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import statistics


In [ ]:
# # Download data (run only once)
# !wget https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip
# !unzip data.zip -d data

In [2]:
# Reproducibility settings
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [3]:
# CNN Model
class HairTypeCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=0)
        self.pool = nn.MaxPool2d(2, 2)
        flattened = 32 * 99 * 99
        self.fc1 = nn.Linear(flattened, 64)
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = HairTypeCNN().to(device)
model

HairTypeCNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=313632, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=1, bias=True)
)

In [4]:
# Parameter count
total_params = sum(p.numel() for p in model.parameters())
print("Total parameters:", total_params)

Total parameters: 20073473


In [ ]:
# Transforms
train_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ) # ImageNet normalization
])

val_transforms = train_transforms

train_dataset = datasets.ImageFolder("data/data/train", transform=train_transforms)
val_dataset = datasets.ImageFolder("data/data/test", transform=val_transforms)

train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=20, shuffle=False)


In [6]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(model.parameters(), lr=0.002, momentum=0.8)

In [7]:
# Training loop (first 10 epochs)
history = {"acc": [], "loss": [], "val_acc": [], "val_loss": []}

for epoch in range(10):
    model.train()
    running_loss, correct, total = 0, 0, 0

    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        y = y.float().unsqueeze(1)

        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * x.size(0)
        preds = (torch.sigmoid(out) > 0.5).float()
        total += y.size(0)
        correct += (preds == y).sum().item()

    train_loss = running_loss / len(train_dataset)
    train_acc = correct / total
    history["loss"].append(train_loss)
    history["acc"].append(train_acc)

    # Validation
    model.eval()
    val_loss_sum, val_correct, val_total = 0, 0, 0

    with torch.no_grad():
        for x, y in val_loader:
            x, y = x.to(device), y.to(device)
            y = y.float().unsqueeze(1)

            out = model(x)
            loss = criterion(out, y)

            val_loss_sum += loss.item() * x.size(0)
            preds = (torch.sigmoid(out) > 0.5).float()
            val_total += y.size(0)
            val_correct += (preds == y).sum().item()

    val_loss = val_loss_sum / len(val_dataset)
    val_acc = val_correct / val_total
    history["val_loss"].append(val_loss)
    history["val_acc"].append(val_acc)

    print(f"Epoch {epoch+1}: Loss={train_loss:.4f}, Acc={train_acc:.4f}, ValLoss={val_loss:.4f}, ValAcc={val_acc:.4f}")


Epoch 1: Loss=0.6462, Acc=0.6362, ValLoss=0.6032, ValAcc=0.6517
Epoch 2: Loss=0.5475, Acc=0.7100, ValLoss=0.7251, ValAcc=0.6318
Epoch 3: Loss=0.5533, Acc=0.7250, ValLoss=0.5991, ValAcc=0.6716
Epoch 4: Loss=0.4802, Acc=0.7712, ValLoss=0.6033, ValAcc=0.6567
Epoch 5: Loss=0.4334, Acc=0.8025, ValLoss=0.6196, ValAcc=0.6766
Epoch 6: Loss=0.3740, Acc=0.8325, ValLoss=0.7371, ValAcc=0.6766
Epoch 7: Loss=0.2721, Acc=0.8838, ValLoss=0.9223, ValAcc=0.6418
Epoch 8: Loss=0.2478, Acc=0.9000, ValLoss=0.7294, ValAcc=0.7214
Epoch 9: Loss=0.2075, Acc=0.9200, ValLoss=0.7523, ValAcc=0.7015
Epoch 10: Loss=0.1494, Acc=0.9450, ValLoss=0.7894, ValAcc=0.7015


In [8]:
median_acc = statistics.median(history["acc"])
std_loss = statistics.stdev(history["loss"])

print("Median Training Accuracy:", median_acc)
print("Std Dev Training Loss:", std_loss)

Median Training Accuracy: 0.8175
Std Dev Training Loss: 0.1675664032629967


In [9]:
# Augmented transforms
train_transforms_aug = transforms.Compose([
    transforms.RandomRotation(50),
    transforms.RandomResizedCrop(200, scale=(0.9, 1.0), ratio=(0.9, 1.1)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

train_dataset_aug = datasets.ImageFolder("data/data/train", transform=train_transforms_aug)
train_loader_aug = DataLoader(train_dataset_aug, batch_size=20, shuffle=True)


In [10]:
history_aug = {"acc": [], "loss": [], "val_acc": [], "val_loss": []}

for epoch in range(10):
    model.train()
    run_loss, correct, total = 0, 0, 0

    for x, y in train_loader_aug:
        x, y = x.to(device), y.to(device)
        y = y.float().unsqueeze(1)

        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()

        run_loss += loss.item() * x.size(0)
        preds = (torch.sigmoid(out) > 0.5).float()
        total += y.size(0)
        correct += (preds == y).sum().item()

    train_loss = run_loss / len(train_dataset_aug)
    train_acc = correct / total
    history_aug["loss"].append(train_loss)
    history_aug["acc"].append(train_acc)

    # Validation
    model.eval()
    vloss_sum, vcorrect, vtotal = 0, 0, 0

    with torch.no_grad():
        for x, y in val_loader:
            x, y = x.to(device), y.to(device)
            y = y.float().unsqueeze(1)

            out = model(x)
            loss = criterion(out, y)

            vloss_sum += loss.item() * x.size(0)
            preds = (torch.sigmoid(out) > 0.5).float()
            vtotal += y.size(0)
            vcorrect += (preds == y).sum().item()

    vloss = vloss_sum / len(val_dataset)
    vacc = vcorrect / vtotal
    history_aug["val_loss"].append(vloss)
    history_aug["val_acc"].append(vacc)

    print(f"Epoch {epoch+1}: Loss={train_loss:.4f}, Acc={train_acc:.4f}, ValLoss={vloss:.4f}, ValAcc={vacc:.4f}")


Epoch 1: Loss=0.7418, Acc=0.6288, ValLoss=0.6577, ValAcc=0.6816
Epoch 2: Loss=0.5797, Acc=0.6850, ValLoss=0.7317, ValAcc=0.6766
Epoch 3: Loss=0.5754, Acc=0.7063, ValLoss=0.5766, ValAcc=0.6965
Epoch 4: Loss=0.5515, Acc=0.7238, ValLoss=0.5693, ValAcc=0.7065
Epoch 5: Loss=0.5610, Acc=0.7075, ValLoss=0.6311, ValAcc=0.6667
Epoch 6: Loss=0.5328, Acc=0.7288, ValLoss=0.5863, ValAcc=0.7065
Epoch 7: Loss=0.5273, Acc=0.7250, ValLoss=0.7732, ValAcc=0.5970
Epoch 8: Loss=0.5027, Acc=0.7600, ValLoss=0.8391, ValAcc=0.6318
Epoch 9: Loss=0.5065, Acc=0.7362, ValLoss=0.5434, ValAcc=0.7214
Epoch 10: Loss=0.4735, Acc=0.7650, ValLoss=0.5352, ValAcc=0.7413


In [11]:
mean_test_loss = sum(history_aug["val_loss"]) / len(history_aug["val_loss"])
last5_acc = history_aug["val_acc"][5:]
avg_last5 = statistics.mean(last5_acc)

print("Mean Test Loss (Aug):", mean_test_loss)
print("Avg Test Acc (Epochs 6–10):", avg_last5)


Mean Test Loss (Aug): 0.644363372910082
Avg Test Acc (Epochs 6–10): 0.6796019900497512
